A explicação de como foi feita a análise é dividida em 9 - tópicos

1 - Business challenge

2 - Data review

3 - Data processing 

4 - DNN Model building

5 - DNN Model evaluation

6 - Decision Tree

7 - Random forest

8 - Sampling

9 - Summary




1 - Considerações Iniciais: 

Primeiro é importante notar que o dataset está bem desbalanceado, isso pode ter gerado erros ao análisar os dados, dentro de 284.807 transações há somente 492 fraudes, sendo fraudes positivas somente 0.17% do dataset

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("C:/Users/guilh/Documents/GitHub_clone/deteccao-de-fraude/archive/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


2 - Olhando os Dados:

As colunas V1 - V28 são todas obtidas por meio de transformações feitas usando o PCA, com isso temos somente duas colunas que permanecem inalteradas "Amount" e "Time", sendo a coluna Class uma variável de Bernoulli(Estudando PE), com valores 0 para Não fraude, e 1 para Fraude

In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df["AmountStandart"] =  scaler.fit_transform(df["Amount"].values.reshape(-1,1))

3 - Preprocessamento:

- Como o "Amount" tem um range muito grande de dados indo de 0 -  25.691,16, realizamos uma "Standartization" para que todos os dados tenham média 0, e desvio padrão = 1, isso é importante para o uso de redes neurais, que depende da distância entre os dados para realizar a classificação.

- Passos da Standartization abaixo:

Foram extraidos os valores dentro de "Amount", mudamos a dimensionalidade para a de um array, após isso aplicamos a Standartization e criamos a nova coluna no dataframe original com os dados.  

In [3]:
df = df.drop(["Amount","Time"], axis=1)

y = df["Class"]
X = df.drop("Class", axis= 1)

Dropamos as colunas "Amount" e "Time" pq elas não são necessárias para o modelo, com isso temos os valores y, que desejamos prever(Se ocorreu fraude ou não) e os valores x que são as transformações feitas com o PCA

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,train_size =0.7, random_state=42)

Os dados são separados para o Treino e Teste:
(Parte dos dados usamos para treino e parte para teste, pois o modelo não pode testar sua acurácia com dados que já conhece)

In [6]:
import keras
from keras import layers
from keras import ops

model = keras.Sequential(
    [
        layers.Dense(input_dim = X_train.shape[1], units = 16, activation = "relu", name = "layer1"),
        layers.Dense(units = 24, activation = "relu", name = "layer2"),
        layers.Dropout(0.5),
        layers.Dense(units = 20, activation = "relu", name = "layer3"),
        layers.Dense(units = 24, activation = "relu", name = "layer4"),
        layers.Dense(units = 1, activation = "sigmoid", name = "layer5")
    
])

4 - DNN Model building: (Os termos em negrito são explicados no final)
Criação de uma rede neural **Sequencial:**

**Variáveis:**
- **input_dim**: Especifica quantos parâmetros serão recebidos.
- **units**: Número de neurônios.
- **activation("relu")**: Define que para essa camada será usada a função **ReLU (Rectified Linear Unit)**.
- **activation("Sigmoid")**: Define que a saída da DNN será um valor entre 0 e 1, sendo apropriado pois queremos prever a coluna "Classe".

**Comandos:**
- **keras.Sequential**: Cria um modelo **Sequencial** para a criação da rede neural.
- **layer.Dense**: Cria uma camada para realizar o processamento de dados.
- **Dropout**: Desativa aleatoriamente 50% dos neurônios para prevenir **Overfitting**.

**Explicações:**
- **DNN Sequencial**: É uma rede neural onde cada camada é empilhada em cima da anterior. Ou seja, o output da primeira é o input da segunda até atingirmos a última camada, por isso é chamada de Sequencial. Esse tipo de rede neural pode ser separado em três tipos:

  - **Camadas de Entrada**: É a camada inicial que recebe os dados de input, e o número de neurônios nessa camada é igual ao número de features nos dados passados.

  - **Camadas Ocultas**: São as camadas onde são feitas as transformações de redes neurais, como a função de ativação (**Activation Function**), seleção de vieses (**Bias Selection**), multiplicação de pesos.

  - **Camadas de Saída**: Fornece o resultado final da rede neural. O número de neurônios e a função de ativação dessa camada dependem do tipo de dados. Nesse caso, usamos o **Sigmoid**, por ser uma distribuição binomial.

- **Non Linear**: Porque funções não lineares são essenciais para redes neurais?

  Funções Não Lineares são importante devido a sua capacidade de modelar qualquer tipo de função, pois funções lineares somente podem representar outras funções lineares quando somados os neuronios que contêm elas, diferente das não lineares, isso é essêncial, pois o verdadeiro objetivo da rede neural é mapear a função que forma o nosso conjunto de dados a serem previstos, com a função em maõs conseguimos achar os valores ao longo dessa função, sendo que os neurônios podem ser defindos matématicamente como 
  
  $$N(a) = \max(w \cdot a + b, 0)$$

  onde:
- \(max\) é a função **ReLU(Rectified Linear Unit)**
- \(N(a)\) é a saída do neurônio.
- \(w\) é o peso.
- \(a\) é a entrada.
- \(b\) é o bias.
  
  - **Non Linear Activation Function**: São funções matemáticas aplicadas no output de cada neurônio, permitindo que a rede neural aprenda e entenda relações complexas entre os dados. Elas introduzem a não linearidade necessária para o funcionamento das redes neurais.

  - **Sigmoid**: É uma **Non Linear Activation Function** para redes neurais. Os valores de input são transformados em valores entre 0 e 1 no output. Ou seja, valores muito grandes são transformados em 1, enquanto valores pequenos são transformados em 0. Essa função é muito sensível apenas no intervalo próximo de 0.5, pois nesse intervalo é crucial o valor para determinar se é 0 ou 1, enquanto em outros intervalos, se for muito grande é 1 e se for muito pequeno é 0.
  
  - **Rectified Linear Unit (ReLU)**: É uma **Non Linear Activation Function** que parece e funciona como uma **Linear Function**. Mas, como uma função pode ser linear e não linear ao mesmo tempo? A explicação desse modelo pode ser resumida a 4 linhas de código:
    ```python
    if input > 0:
        return input
    else:
        return 0
    ```
    Portanto, quando o input tem valor maior que 0, a função é linear, pois o output será o próprio input. Quando o valor do input é negativo, o output é 0.

- **Overfitting**: É um problema comum em problemas de previsão de dados. Ocorre quando o modelo aprende muito bem os dados de treino, mas, quando adicionamos novos dados para teste, sua performance cai drasticamente. As principais causas para esse problema são: poucos dados para realizar o treinamento do modelo, modelos muito complexos que aprenderam muito bem os parâmetros nos dados de treino e não conseguem interpretar dados diferentes, ruídos nos dados de treinamento (como outliers) fazem com que parâmetros que não existem sejam observados, diminuindo assim a acurácia do teste.

Para melhor entendimento do funcionamento das funções de ativação:
[Rectified Linear Activation Function for Deep Learning Neural Networks](https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/)

PS: Se vc não quiser ler toda a documentação veja esse vídeo PORFAVOR!:
[Why Neural Networks Must Be Nonlinear](https://www.youtube.com/watch?v=0QczhVg5HaI)

Playground util para a melhor visualização ao contruir uma rede neural:
[Playground_DNN](https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.43936&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false)

In [15]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 15, epochs = 5)
score = model.evaluate(X_test, y_test)
print(score)

Epoch 1/5
13291/13291 ━━━━━━━━━━━━━━━━━━━━ 10s 698us/step - accuracy: 0.9987 - loss: 0.0147
Epoch 2/5
13291/13291 ━━━━━━━━━━━━━━━━━━━━ 9s 690us/step - accuracy: 0.9992 - loss: 0.0043
Epoch 3/5
13291/13291 ━━━━━━━━━━━━━━━━━━━━ 10s 724us/step - accuracy: 0.9994 - loss: 0.0036
Epoch 4/5
13291/13291 ━━━━━━━━━━━━━━━━━━━━ 10s 714us/step - accuracy: 0.9994 - loss: 0.0038
Epoch 5/5
13291/13291 ━━━━━━━━━━━━━━━━━━━━ 9s 708us/step - accuracy: 0.9993 - loss: 0.0042
2671/2671 ━━━━━━━━━━━━━━━━━━━━ 1s 513us/step - accuracy: 0.9993 - loss: 0.0039
[0.003038892988115549, 0.9993211627006531]


5. DNN model evaluation (Os termos em negrito são explicados no final):

**Variáveis**:
- **optimizer**: Define qual o algoritmo que o modelo irá usar para otimização ao modificar os pesos, sendo o **adam**.
- **loss**: Especifica a função de classificação que irá avaliar como se saiu o modelo durante o treino, sendo essa **binary_crossentropy**.
- **metrics**: Mostra qual métrica será usada para classificar o modelo durante o treino e teste, sendo a acurácia dada em %.
- **batch_size**: Valor que demonstra quantas amostras serão propagadas (**Back Propagation**) antes de atualizar os pesos do modelo.
- **epochs**: O número de vezes que o modelo será treinado.

**Comandos**:
- **model.compile**: Configura como o modelo irá otimizar, classificar e metrificar a acurácia classificada.
- **model.fit**: Comando usado para passarmos os dados para o modelo, no caso a rede neural sequencial criada acima.
- **model.evaluate**: Calcula a **loss** e as métricas para acurácia do modelo.

**Explicações**:
- **loss**: É o parâmetro usado para a classificação da entropia binária cruzada e representa o quanto um modelo conseguiu se ajustar aos dados de treino.


- **Gradiente Descendente**: É um algoritmo usado para encontrar os mínimos de uma função. Calculamos a derivada da função **loss** (função de perda) e queremos que essa função derivada atinja seu valor mínimo, para que a perda dos parâmetros(weight,bias) associados à função **loss** seja mínima. Quando atingimos o mínimo da derivada **loss**, isso significa que encontramos os parâmetros que fazem com que o modelo tenha a menor perda entre os dados previstos e os dados reais. Assim, os pesos e o vies são atualizados para se enquadrar com os parâmetros de minimização da função **loss** encontrados.

  **Como chegamos ao mínimo da função derivada da **loss**?**:
  - Realizamos uma comparação entre a distância dos valores reais e a média móvel dos valores previstos. Essa distância é somada de 1 até os n parâmetros "a". Após isso, derivamos a soma e multiplicamos ela pelo learning_rate ("n") passado dentro do *optimizer*, e será subtraída essa multiplicação pelo valor do parâmetro "a" até que seja atingido o valor mínimo da derivada.

  \[
  a = a − η ⋅ ∇J(a)
  \]

  - **Exemplo (É complicado, então aqui vai um exemplo):**
    - Uma função \( J(a) = a^2 \), sua derivada é \( 2a \). Com isso, calculamos o gradiente descendente. Esse cálculo é feito m vezes até chegar no mínimo:
    \[
    a = a - η ⋅ 2a
    \]

**Gradientes Descendentes Estocásticos**: É uma variação do algoritmo **Gradiente Descendente**, utilizado para minimizar a **loss** das redes neurais atualizando os parâmetros do modelo em cada uma das **epochs** do modelo, fazendo com que converja mais rapidamente.

**adam**: Modelo de otimização de pesos que combina as vantagens dos modelos **AdaGrad** e **RMSProp**. Calcula os **Gradientes Descendentes Estocásticos**, computando a média móvel dos gradientes, depois a média móvel do quadrado dos gradientes. Com base nos resultados, calcula o Bias e atualiza os pesos da rede neural.

**binary_crossentropy**: Classificação binária de entropia cruzada, mede a diferença entre a distribuição predita e a distribuição real das classes, sendo ideal para distribuições binárias.

**Back Propagation**: Utiliza o método de otimização dos **Gradientes Descendentes Estocásticos** para minimizar a perda. Após a saída das redes neurais, esse algoritmo roda propagando o erro calculado pelo **binary_crossentropy** de volta para a rede camada por camada, usando os **Gradientes Descendentes Estocásticos** para atualizar os pesos de cada camada. Essa propagação é feita igual ao número de vezes especificado por **epochs**.

Para aprofundar em Gradiente Descendente leia:
[Gradientes Descendentes na Prática - Melhor Jeito de Entender](https://medium.com/data-hackers/gradientes-descendentes-na-prática-melhor-jeito-de-entender-740ef4ff6c43)



--------------------------------------------------------------------------------------------------------------------------------------------------

#**Acurrácia**:

Como haviamos determinado nossa metrica como acurrácia, o output do modelo foi em porcentagem o problema agora é com a falta de balanceamento dos dados, 
que faz com que nosso modelo tenha uma boa acurrácia, mas esse medida é falha, porque como nós temos poucas fraudes(ex: 5%), caso nosso modelo somente
 preveja tudo como "não fraude" ainda terá uma acurrácia de 95%, mas não representa realmente a sua eficácia em um cenário real, com isso usamos uma 
 matriz de erro chamada de **Confusion Matrix** para calcular a performance, com isso temos:

**Confusion Matrix**: É uma tabela que permite a visualização melhor da performance de um algoritmo, sendo muito util para comparar qual metódo é melhor,
assim como feito nesse estudo, no qual comparamos DNNs com DecisionTree, RandomForest entre outras, a construção da matriz é simples, as linhas 
indicam o valor da variável aleatória(Se for o Daniel lendo são eventos) para os valores reais do dataset, enquanto a coluna representa a mesma variável 
aleatória(evento) mas para as previsões feitas pelo modelo, com isso os valores representados são respectivamente: 

    Verdadeiros-Negativo  Falsos-Positivos
    Falso-Negativo       Verdadeiro-Positivo

- DNN mostram uma precisão de 85.3%, Sensibilidade 78.9%, e F1 score de 82.0%.


![Confusion_Matrix_DNN](../assets/images/Confusion_matrix_1.png)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train, y_train)
y_pred = decision_tree_model.predict(X_test)

6. Decision Tree:

Precisamos explicar o que são **"DecisionTreeClassifier"**

*"Comandos":*

 - **"decision_tree_model.predict"** = Utiliza o modelo de árvore de decisão para prever os dados em "y_test" com base nos dados em X_test


*"Explicações":*

 - **"DecisionTreeClassifier"** = É uma classe que é usada para a previsão de dados a partir de caracteristicas especificas observados pelos modelos, sendo que são feitas divisões recursivas(igual árvore de recursão em APA) essas divisões usam os caracteristicas especificas observados, os dados irão descer a esquerda ou direita da árvore, dependendo de serem verdadeiros ou falsos para determinado paramêtro.


**Resultados:**:

Ao utilizar a matriz de confusão para metrificar como o modelo performou as arvores de decisão mostram uma precisão de 82.7%, Sensibilidade 74.8%, e F1 score de 78.6%.

Foram piores que as Redes Neurais Sequênciais

![Confusion_Matrix_Arvores](..\assets\images\Confusion_matrix_2.png)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators = 100)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

7. Random forest:

*"Variáveis":*

- **"n_estimators"** = Define quantas árvores de decisão serão criadas


*"Explicações":*

 - **"RandomForestClassifier"** = As Florestas de classificação aleátorias são uma expanção da árvore de decisão, pois representam várias arvores de decisão, as quais utilizam parte do conjunto de dados para aprendender diferentes características, além de que podem reutilizar dados, como não são utilizados todo o conjunto de dados em casa árvore são selecionadas partes do dataset para cada uma, as arvores são criadas sequencialmente para conseguir corrigir o erro das 
 
**Resultados:**:

Precisão de 95%, Sensibilidade de 77.5%, e score F1 de 85.3%, 

Foram melhores que as Redes Neurais Sequenciais


![Confusion_Matrix_Florestas](../assets/images/Confusion_matrix_3.png)

In [10]:
num_frauds = len(df[df.Class == 1])
under_sample_ind = df[df.Class == 0].sample(num_frauds).index
fraud_ind = np.array(df[df.Class == 1].index)
normal_ind = df[df.Class == 0].index
random_normal_ind = np.random.choice(normal_ind, num_frauds, replace
= False)
random_normal_ind = np.array(random_normal_ind)
under_sample_data = df.iloc[under_sample_ind, :]

#É feito um Under-Sampling dos dados

X_undersample = under_sample_data.iloc[:, under_sample_data.columns != "Class"]
y_undersample = under_sample_data.iloc[:, under_sample_data.columns == "Class"]

8. Sampling:

*"Variáveis":*

- **"num_frauds"** = A soma do número de fraudes dentro da colunas "Class".

- **"under_sample_if"** = Recebe a mesma quantidade de linhas que num_frauds possuem, mas nesse caso sendo as que não são fraude.

- **"fraud_ind"** = Representa um array que tenha como indice o número de fraudes.

- **"normal_ind"** = Representa um array que tenha como indice o número de não-fraudes.

- **"random_normal_ind"** = Escolhe uma aleatóriamente fraudes com relação ao número de operações normais, garantindo que a quantidade de linhas é 
a mesma, depois recebe as fraudes como um array.

- **"under_sample_data"** =  Cria um novo dataframe com base no df anterior selecionando somente as linhas especificadas dentro de "under_sample_ind".

- **"X_undersample"** = junta dentro de um array X os valores de PCA, que não sejam o de classe a serem previstos, mais esse novo array so será composto de valores selecionados igual ao número de linhas de fraudes.

- **"Y_undersample"** = junta dentro de um array Y os valores de Classe, a serem previstos, mais esse novo array so será composto de valores selecionados igual ao número de linhas de fraudes.

    
*"Comandos":*

- **"iloc"** = Comando usado dentro do pandas para selecionar e filtrar colunas especificamente

- **"sample"** = Seleciona uma porcentagem de dados dentro do dataset original, caso não seja especificado seleciona a quantidade de linhas da varável passada, no caso num_frauds


*"Explicações":*

- **"Sampling"** = Tecnica usada para lidar com a falta de balanceamento nos dados, se refere a selecionar um subset de dados menor dentro do dataset, realizar o "train_test_split" é um tipo de sampling, exemplos disso podem ser, selecionar aléatoriamente uma quantidade de dados dentro do dataset, criar sinteticamente dados para que o balanceamento seja feito uma maneira melhor, retirar dados do conjunto majoritario para que o conjunto menor tenha mais impacto, selecionar uma porcentagem de dados que represente a amostra como um todo


8.1 Under-sampling:


*"Explicações":*

- **"Under-Sampling"** = É um tipo de separação do conjunto de dados, sendo que nesse caso é feita retirando a parte que contém mais dados, ou seja, quando não é fraude, para que as partes que contenham fraude sejam mais substânciais na análise feita pelo modelo


**Resultados:**:

Reetreinamos o modelo de redes neurais com os dados após o sampling teve Precisão de 100%, Sensibilidade de 87.6%, e score F1 de 93.4%, 

![Confusion_Matrix_Under_Sampling](../assets/images/Confusion_matrix_4.png)

In [21]:
from imblearn.over_sampling import SMOTE

X_resample, y_resample = SMOTE().fit_resample(X, y)

8.2 SMOTE

*"Explicações":*

-**"Smote"** = Synthetic Minority Over-sampling é uma das tecnicas de realizar Sampling, são criados dados sintéticos para os dados em minoria, fazendo com que o dataset fique mais balanceado.

**Resultados:**:

Para o modelo de DNN Precisão de 99.8%, Sensibilidade de 100%, e score F1 de 99.9%.

![Confusion_Matrix_Over_Sampling](../assets/images/Confusion_matrix_5.png)

Como questão de curiosidade o dataset não mostra quando retreinamos os modelos após o Under-Sampling e o Over-Sampling para os meus dados, assim

**Under-Sampling**:

In [17]:
import keras
from keras import layers
from keras import ops

num_frauds = len(df[df.Class == 1])
under_sample_ind = df[df.Class == 0].sample(num_frauds).index
fraud_ind = np.array(df[df.Class == 1].index)
normal_ind = df[df.Class == 0].index
random_normal_ind = np.random.choice(normal_ind, num_frauds, replace
= False)
random_normal_ind = np.array(random_normal_ind)
under_sample_data = df.iloc[under_sample_ind, :]

#É feito um Under-Sampling dos dados

X_undersample = under_sample_data.iloc[:, under_sample_data.columns != "Class"]
y_undersample = under_sample_data.iloc[:, under_sample_data.columns == "Class"]

model = keras.Sequential(
    [
        layers.Dense(input_dim = X_undersample.shape[1], units = 16, activation = "relu", name = "layer1"),
        layers.Dense(units = 24, activation = "relu", name = "layer2"),
        layers.Dropout(0.5),
        layers.Dense(units = 20, activation = "relu", name = "layer3"),
        layers.Dense(units = 24, activation = "relu", name = "layer4"),
        layers.Dense(units = 1, activation = "sigmoid", name = "layer5")
    
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_undersample, y_undersample, batch_size = 15, epochs = 5)
score = model.evaluate(X_undersample, y_undersample)
print(score)



Epoch 1/5


c:\Users\guilh\Documents\GitHub_clone\deteccao-de-fraude\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 875us/step - accuracy: 0.2483 - loss: 0.8368
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - accuracy: 0.9855 - loss: 0.4780
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - accuracy: 1.0000 - loss: 0.2034
Epoch 4/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - accuracy: 1.0000 - loss: 0.0555
Epoch 5/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 1.0000 - loss: 0.0172
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 1.0000 - loss: 0.0046
[0.004465003963559866, 1.0]


**Over-Sampling**:

In [20]:
import keras
from keras import layers
from keras import ops
from imblearn.over_sampling import SMOTE


X_resample, y_resample = SMOTE().fit_resample(X, y)


model = keras.Sequential(
    [
        layers.Dense(input_dim = X_resample.shape[1], units = 16, activation = "relu", name = "layer1"),
        layers.Dense(units = 24, activation = "relu", name = "layer2"),
        layers.Dropout(0.5),
        layers.Dense(units = 20, activation = "relu", name = "layer3"),
        layers.Dense(units = 24, activation = "relu", name = "layer4"),
        layers.Dense(units = 1, activation = "sigmoid", name = "layer5")
    
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 15, epochs = 5)
score = model.evaluate(X_resample, y_resample)
print(score)



Epoch 1/5


c:\Users\guilh\Documents\GitHub_clone\deteccao-de-fraude\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13291/13291 ━━━━━━━━━━━━━━━━━━━━ 10s 696us/step - accuracy: 0.9958 - loss: 0.0223
Epoch 2/5
13291/13291 ━━━━━━━━━━━━━━━━━━━━ 9s 691us/step - accuracy: 0.9994 - loss: 0.0035
Epoch 3/5
13291/13291 ━━━━━━━━━━━━━━━━━━━━ 9s 697us/step - accuracy: 0.9993 - loss: 0.0039
Epoch 4/5
13291/13291 ━━━━━━━━━━━━━━━━━━━━ 9s 695us/step - accuracy: 0.9994 - loss: 0.0036
Epoch 5/5
13291/13291 ━━━━━━━━━━━━━━━━━━━━ 9s 688us/step - accuracy: 0.9994 - loss: 0.0034
17770/17770 ━━━━━━━━━━━━━━━━━━━━ 9s 500us/step - accuracy: 0.9729 - loss: 0.1690
[0.5477353930473328, 0.912932813167572]
